In [1]:
import trainHelper
import utils
from mds.lmds import landmarkMDS
from mds.cmds import classicalMDS
from mds.fastmap import fastmap
import numpy
from torch.utils.data import DataLoader
import torch
from torch import nn, Tensor
import lossFunction as lossF
from model.AutoEncoder import AutoEncoder
from model.DynParam import DynParam
from model.Linear import Linear
from model.VAE import VAE
import os.path

In [ ]:
import sys
from importlib import reload
reload(sys.modules['utils']);
reload(sys.modules['trainHelper']);
reload(sys.modules['lossFunction']);

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')


In [3]:

sample_space = (1000000, 100)
ss, N, d = 1000, 15, 2


In [6]:

try:
    euclidean_data1 = utils.load_variable('data/euclidean_data1.pkl')
    euclidean_data2 = utils.load_variable('data/euclidean_data2.pkl')
    
    rand_data1 = utils.load_variable('data/rand_data.pkl')
    rand_data2 = utils.load_variable('data/rand_data.pkl')
    
    if euclidean_data.size() != (ss, 1, N, N):
        print("Updated data for requirement !")
        raise Exception("Previous data not match requirement !")

except:
    euclidean_data1 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    euclidean_data2 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data1 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data2 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    utils.dump_variable(euclidean_data1, 'data/euclidean_data1.pkl')
    utils.dump_variable(euclidean_data2, 'data/euclidean_data2.pkl')

    utils.dump_variable(rand_data1, 'data/rand_data1.pkl')
    utils.dump_variable(rand_data2, 'data/rand_data2.pkl')

In [7]:
data = torch.stack([
            euclidean_data1,
            euclidean_data2,
            # rand_data1,
            # rand_data2,
        ]).view(ss * 2, 1, N, N)

data = torch.tensor(utils.vectorize_distance_from_DM(data).data, requires_grad=True)
# data = torch.tensor(data.view(ss * 2, 1, N * N).data, requires_grad=True)

# data = torch.tensor(euclidean_data.view(ss, 1, N * N).data, requires_grad=True)

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

In [17]:
test_batch = 200
test_data = utils.generate_rand_DM(
                N=N,
                sample_size=test_batch, 
                sample_space=sample_space, isInt=True, 
                v_size=2, 
                transform_func=lambda x: x[0] ** 2 + x[1])

In [26]:
def test_model():

    t_data = torch.tensor(utils.vectorize_distance_from_DM(test_data).data, requires_grad=True)
    
    model_rs = helper.model(t_data)
    
    model_dm = torch.stack(
        [utils.unvectorize_distance(rs, N).view(1, N, N) for rs in model_rs])

    return torch.mean((model_dm - d) ** 2)

print("Test Result: ", test_model())

Test Result:  tensor(3.5483, grad_fn=<MeanBackward1>)


In [27]:
for i in [32, 64, 128, 256]:

    model_id = "LinearAE_with_two_" + str(i) + "_layers_distance"

    in_dim = data[0].size()[-1]
    out_dim = N * 2

    model = AutoEncoder([in_dim, i, i, out_dim], final_activation=nn.Sigmoid)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    lossFun = nn.MSELoss(reduction='mean')

    helper = trainHelper.TrainHelper(
        id=model_id,
        model=model,
        optimizer=optimizer,
        lossFun=lossFun,
        lr_factor=0.1)


    EPOCH = 300
    print_on_each = 100

    time_used = utils.time_measure(helper.train, 
                        [dlr, EPOCH, print_on_each])[1]

    print("Time used for the training: ", time_used, "s")

    helper.backup()
    
    print("Test Result: ", test_model())


0 	| Mean loss: 0.05966806314385117
100 	| Mean loss: 0.048000652947386147
200 	| Mean loss: 0.04756927491769634
Time used for the training:  1960.020983 s
Test Result:  tensor(3.5022, grad_fn=<MeanBackward1>)
0 	| Mean loss: 0.05945083094780311
100 	| Mean loss: 0.04926439288786694
200 	| Mean loss: 0.04872649582968837
Time used for the training:  2005.70552 s
Test Result:  tensor(3.3713, grad_fn=<MeanBackward1>)
0 	| Mean loss: 0.06009331275626099
100 	| Mean loss: 0.05619122153769714
200 	| Mean loss: 0.056173837739133724
Time used for the training:  2211.547043 s
Test Result:  tensor(3.0492, grad_fn=<MeanBackward1>)


In [ ]:
EPOCH = 200
print_on_each = 10

time_used = utils.time_measure(helper.train, 
                    [dlr, EPOCH, print_on_each])[1]

print("Time used for the training: ", time_used, "s")

helper.backup()

In [ ]:
helper.merge_local_record()

In [ ]:
helper.plot(['loss_mean', 'loss_max', 'loss_min'], value_label='loss')
helper.plot(['train_time'], value_label='train_time')
helper.plot(['lr'], value_label='lr')

In [43]:

cmds_loss, fastmap_loss, model_loss = [], [], []
recon_loss = []

with torch.no_grad():

    for d in test_data:

        d1 = numpy.array(d[0].data)

        cmds_rs = classicalMDS(d1, 2)
        cmds_rs = torch.tensor(cmds_rs)
        cmds_dm, _ = utils.get_distance_matrix(cmds_rs)
        cmds_loss.append(torch.sum((cmds_dm - d)** 2))

        fastmap_rs = fastmap(d1, 2)
        fastmap_rs = torch.tensor(fastmap_rs)
        fastmap_dm, _ = utils.get_distance_matrix(fastmap_rs)
        fastmap_loss.append(torch.sum((fastmap_dm - d)** 2))

        d2 = utils.vectorize_distance_from_DM(d)
        model_rs = helper.model.encode(d2).view(1, N, 2)
        # model_dm = utils.unvectorize_distance(model_rs, N).view(1, N, N) # utils.get_distance_matrix(model_rs)

        model_dm, _ = utils.get_distance_matrix(model_rs)
        model_loss.append(torch.sum((model_dm - d) ** 2))

        try:
            recon_dm = helper.model(d2)
            recon_dm = utils.unvectorize_distance(model_rs, N).view(1, N, N)

            d3 = numpy.array(recon_dm[0].data)

            recon_rs = classicalMDS(d3, 2)
            recon_rs = torch.tensor(recon_rs)
            recon_dm, _ = utils.get_distance_matrix(recon_rs)
            recon_loss.append(torch.sum((recon_dm - d)** 2))
        except Exception as e: 
            pass # print(str(e))

    print("cmds_loss: \t", torch.tensor(cmds_loss).mean(), "|" , len(cmds_loss), "success")    
    print("fastmap_loss: \t", torch.tensor(fastmap_loss).mean(), "|" , len(fastmap_loss), "success")    
    print("model_loss: \t", torch.tensor(model_loss).mean(), "|" , len(model_loss), "success")    
    print("recon_loss: \t", torch.tensor(recon_loss).mean(), "|" , len(recon_loss), "success")    


cmds_loss: 	 tensor(71.2871) | 200 success
fastmap_loss: 	 tensor(nan) | 200 success
model_loss: 	 tensor(94.6814) | 200 success
recon_loss: 	 tensor(nan) | 0 success


In [41]:
model_dm

(tensor([[[0.0000, 0.4378, 0.2877, 0.4055, 0.2831, 0.0761, 0.4351, 0.0663,
           0.1717, 0.3291, 0.7714, 0.1441, 0.0653, 0.0559, 0.1177],
          [0.4378, 0.0000, 0.1436, 0.0308, 0.0292, 0.4154, 0.2548, 0.2131,
           0.1566, 0.8453, 0.3693, 0.1154, 0.1806, 0.2552, 0.1227],
          [0.2877, 0.1436, 0.0000, 0.2569, 0.1723, 0.1352, 0.0259, 0.2429,
           0.3019, 0.3184, 0.1280, 0.0352, 0.0914, 0.2792, 0.0542],
          [0.4055, 0.0308, 0.2569, 0.0000, 0.0131, 0.4799, 0.4243, 0.1565,
           0.0780, 1.0000, 0.6024, 0.1644, 0.1976, 0.1884, 0.1565],
          [0.2831, 0.0292, 0.1723, 0.0131, 0.0000, 0.3342, 0.3226, 0.0929,
           0.0505, 0.7877, 0.5118, 0.0868, 0.1090, 0.1202, 0.0795],
          [0.0761, 0.4154, 0.1352, 0.4799, 0.3342, 0.0000, 0.1943, 0.1808,
           0.3209, 0.1109, 0.4307, 0.0929, 0.0642, 0.1865, 0.0901],
          [0.4351, 0.2548, 0.0259, 0.4243, 0.3226, 0.1943, 0.0000, 0.4208,
           0.5046, 0.2880, 0.0481, 0.1186, 0.2010, 0.4653, 0.1508],